In [4]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import shutil

In [3]:
# batch_size=512
# projection_dim=128


# total_results = []

# for task in ['mortality_90days', 'readmission_15days'] :
#     for seed in range(2026, 2029) : 
#         for modality in ['tabular', 'lab', 'note'] :
#             result_dir = Path(f'Results/Linear_{modality}/Seed_{seed}/{task}_proj_{projection_dim}_batch_{batch_size}')
#             exp_dir = result_dir
#             output_dir = exp_dir / 'outputs'
#             model_dir = exp_dir / 'models'
#             model_paths = list(model_dir.glob('linear_epoch*.pth'))
#             # {epoch : model_path}
#             model_paths = {int(p.stem.split('_')[1].replace('epoch', '')) : p for p in model_paths}
            
#             best_output_save_path = output_dir / 'best_epoch.pkl'
#             best_score_save_path = output_dir / 'best_epoch_scores.pkl'
#             best_score =0
#             save = True 
#             for epoch in range(1, 101) : 
#                 epoch_output_path = output_dir / f'epoch{epoch:03d}_scores.pkl'
#                 if not epoch_output_path.exists() :
#                     continue
#                 epoch_output = pickle.load(open(epoch_output_path, 'rb'))
                
#                 valid_output = epoch_output['valid']
#                 valid_score = valid_output['auroc'] + valid_output['auprc']
#                 if valid_score > best_score : 
#                     best_score = valid_score
#                     best_epoch = epoch
#                     best_epoch_output = epoch_output
#                     # print(f"Best Epoch : {best_epoch} with Valid Score : {best_score:.4f} | AUROC : {valid_output['auroc']:.4f}, AUPRC : {valid_output['auprc']:.4f} | TEST AUROC : {epoch_output['test']['auroc']:.4f}, AUPRC : {epoch_output['test']['auprc']:.4f}")
#             if save : 
#                 shutil.copy(output_dir / f'epoch{best_epoch:03d}.pkl', best_output_save_path)
#                 shutil.copy(output_dir / f'epoch{best_epoch:03d}_scores.pkl', best_score_save_path)
#                 model_path = model_paths[best_epoch]
#                 best_model_save_path = model_dir / 'best_model.pth'
#                 shutil.copy(model_path, best_model_save_path)
#                 print(f"Copied best epoch output to {best_output_save_path} & {best_score_save_path}")
#             total_results.append({
#                 'seed' : seed,
#                 'task' : task,
#                 'modality' : modality,
#                 'best_epoch' : best_epoch,
#                 'valid_auroc' : best_epoch_output['valid']['auroc'],
#                 'valid_auprc' : best_epoch_output['valid']['auprc'],
#                 'test_auroc' : best_epoch_output['test']['auroc'],
#                 'test_auprc' : best_epoch_output['test']['auprc'],
#             })
# total_results_df = pd.DataFrame(total_results)
# total_results_df

In [5]:
batch_size=512
projection_dim=128


total_results = []

for task in ['mortality_90days', 'readmission_15days'] :
    for fusion_method in ['Sum', 'WeightedFusion', 'AttnMaskedFusion'] : 
        for seed in range(2026, 2029) : 
            # Results/Linear_MultiModal/Fusion_AttnMaskedFusion/Seed_2026/mortality_90days_proj_128_batch_512/outputs
            result_dir = Path(f'Results/Linear_MultiModal/Fusion_{fusion_method}/Seed_{seed}/{task}_proj_{projection_dim}_batch_{batch_size}')
            exp_dir = result_dir
            output_dir = exp_dir / 'outputs'
            model_dir = exp_dir / 'models'
            
            best_output_save_path = output_dir / 'best_epoch.pkl'
            best_score_save_path = output_dir / 'best_epoch_scores.pkl'
            
            score_paths = list(output_dir.glob('epoch*_scores.pkl'))
            if len(score_paths) == 0 :
                print(f"No output paths for {result_dir}, skipping...")
                continue
            score_paths = sorted(score_paths, key=lambda x: int(x.stem.split('_')[0].replace('epoch', '')))
            # last epoch is the best
            best_epoch_path = score_paths[-1]
            best_epoch = int(best_epoch_path.stem.split('_')[0].replace('epoch', ''))
            best_epoch_score = pickle.load(open(best_epoch_path, 'rb'))
            
            model_paths = list(model_dir.glob('linear_epoch*.pth'))
            # {epoch : model_path}
            model_paths = {int(p.stem.split('_')[1].replace('epoch', '')) : p for p in model_paths}
            model_path = model_paths[best_epoch]
            
            shutil.copy(output_dir / f'epoch{best_epoch:03d}.pkl', best_output_save_path)
            shutil.copy(output_dir / f'epoch{best_epoch:03d}_scores.pkl', best_score_save_path)
            model_path = model_paths[best_epoch]
            best_model_save_path = model_dir / 'best_model.pth'
            shutil.copy(model_path, best_model_save_path)
            print(f"Copied Best Epoch Output {best_epoch_path} \n\t\t\t-> {best_output_save_path}")
            total_results.append({
                'seed' : seed,
                'task' : task,
                'fusion' : fusion_method,
                'best_epoch' : best_epoch,
                'valid_auroc' : best_epoch_score['valid']['auroc'],
                'valid_auprc' : best_epoch_score['valid']['auprc'],
                'test_auroc' : best_epoch_score['test']['auroc'],
                'test_auprc' : best_epoch_score['test']['auprc'],
            })
total_results_df = pd.DataFrame(total_results)
total_results_df

Copied Best Epoch Output Results/Linear_MultiModal/Fusion_Sum/Seed_2026/mortality_90days_proj_128_batch_512/outputs/epoch066_scores.pkl 
			-> Results/Linear_MultiModal/Fusion_Sum/Seed_2026/mortality_90days_proj_128_batch_512/outputs/best_epoch.pkl
Copied Best Epoch Output Results/Linear_MultiModal/Fusion_Sum/Seed_2027/mortality_90days_proj_128_batch_512/outputs/epoch032_scores.pkl 
			-> Results/Linear_MultiModal/Fusion_Sum/Seed_2027/mortality_90days_proj_128_batch_512/outputs/best_epoch.pkl
Copied Best Epoch Output Results/Linear_MultiModal/Fusion_Sum/Seed_2028/mortality_90days_proj_128_batch_512/outputs/epoch069_scores.pkl 
			-> Results/Linear_MultiModal/Fusion_Sum/Seed_2028/mortality_90days_proj_128_batch_512/outputs/best_epoch.pkl
Copied Best Epoch Output Results/Linear_MultiModal/Fusion_WeightedFusion/Seed_2026/mortality_90days_proj_128_batch_512/outputs/epoch022_scores.pkl 
			-> Results/Linear_MultiModal/Fusion_WeightedFusion/Seed_2026/mortality_90days_proj_128_batch_512/outpu

,seed,task,fusion,best_epoch,valid_auroc,valid_auprc,test_auroc,test_auprc
0,2026,mortality_90days,Sum,66,0.854701,0.506358,0.855253,0.501377
1,2027,mortality_90days,Sum,32,0.850201,0.497009,0.854879,0.494888
2,2028,mortality_90days,Sum,69,0.856318,0.515827,0.848346,0.500702
3,2026,mortality_90days,WeightedFusion,22,0.845025,0.478957,0.846865,0.477506
4,2027,mortality_90days,WeightedFusion,78,0.838854,0.475016,0.841844,0.478992
5,2028,mortality_90days,WeightedFusion,18,0.854665,0.502105,0.841604,0.487727
6,2026,mortality_90days,AttnMaskedFusion,7,0.847795,0.495151,0.848037,0.481874
7,2027,mortality_90days,AttnMaskedFusion,35,0.845689,0.486762,0.848141,0.486416
8,2028,mortality_90days,AttnMaskedFusion,47,0.856073,0.510964,0.843540,0.493405
9,2026,readmission_15days,Sum,5,0.682016,0.438639,0.685953,0.439580
